In [ ]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa import stattools

In [ ]:
# Read dataset into a pandas.DataFrame
beer_df = pd.read_csv(
    '../Data files/quarterly-beer-production-in-aus-March 1956-June 1994.csv', index_col=['Quarter']
)

In [ ]:
# Display shape of the dataset
print('Shape of the dataframe:', beer_df.shape)

In [ ]:
# Show top 5 rows
beer_df.head(5)

In [ ]:
# Rename the 2nd column
beer_df.rename(columns={
    'Quarterly beer production in Australia: megalitres. March 1956 ? June 1994': 'Beer_Prod'
}, inplace=True)

In [ ]:
# Remove missing values
missing = (pd.isnull(beer_df.index)) | (pd.isnull(beer_df['Beer_Prod']))
print('Number of rows with at least one missing values:', missing.sum())
beer_df = beer_df.loc[~missing, :]
print('Shape after removing missing values:', beer_df.shape)

In [ ]:
# In order to remove seasonal patterns let us calculate 2X4 quarter moving average
MA4 = beer_df['Beer_Prod'].rolling(window=4).mean()
TwoXMA4 = MA4.rolling(window=2).mean()
TwoXMA4 = TwoXMA4.loc[~pd.isnull(TwoXMA4)]

In [ ]:
# Let's plot the original time series and the seasonal moving averages
fig = plt.figure(figsize=(5.5, 5.5))
ax = fig.add_subplot(1,1,1)
beer_df['Beer_Prod'].plot(ax=ax, color='b', linestyle='-')
TwoXMA4.plot(ax=ax, color='r', linestyle='-')
plt.xticks(rotation=60)
ax.set_title('Quaterly Beer Production between in Australia and 2X4 quarter MA')

### <b> Why?
The original time series on the quarterly beer productions has trend as well as seasonality and therefore is not stationary. Let us see if we can stationarize the time series by first removing the trend component and then taking seasonal differences.

In [ ]:
# Let's compute the residuals after removing the trend
residuals = beer_df['Beer_Prod'] - TwoXMA4
residuals = residuals.loc[~pd.isnull(residuals)]

In [ ]:
# Let's plot the residuals
fig = plt.figure(figsize=(5.5, 5.5))
ax = fig.add_subplot(1,1,1)
residuals.plot(ax=ax, color='b', linestyle='-')
plt.xticks(rotation=60)
ax.set_title('Residuals in Quaterly Beer Production time series')

In [ ]:
# Let's plot the autocorrelation function of the residuals
fig = plt.figure(figsize=(5.5, 5.5))
ax = fig.add_subplot(1,1,1)
autocorrelation_plot(residuals, ax=ax)
ax.set_title('ACF of Residuals in Quaterly Beer Production time series')
autocorrelation_plot(residuals)

In [ ]:
# Perform Ljung-Box test on residuals to get the p-values
# We will use lags of upto 20

# acceptance of null hypothesis confirms stationarity of the time series.

# p-val > alpha Null hypothesis is not rejected means stationary series
# p-val < alpha Null hypothesis is rejected means non-stationary series
_, _, _, pval_residuals = stattools.acf(
    residuals, unbiased=True, nlags=20, qstat=True, alpha=0.05
)
print('Null hypothesis is rejected for lags:', np.where(pval_residuals<=0.05))

In [ ]:
# Perfom ADF test and check stationary for residual
adf_result = stattools.adfuller(residuals, autolag='AIC')
# alpha is the probability of rejecting the null hypothesis, if it is true.
# p-val > alpha Null hypothesis is not rejected means non-stationary series
# p-val < alpha Null hypothesis is rejected means stationary series

# The null hypothesis is NOT rejected as the p-value is greater than 0.05.

print('p-val of the residuals:', adf_result[1])

#### <b> Summary table:

<table>
<tr>
    <td> 
        <b>Tests</b>
    </td>
    <td> 
        <b>Augmented Dickey-Fuller (ADF)</b>
    </td>
    <td> 
        <b>Ljung-Box</b>
    </td>
    <td> 
        <b>p-values</b>
    </td>
</tr>

<tr>
    <td> 
    Null Hypothesis
    </td>
    <td> 
    Non Stationary
    </td>
    <td> 
    Stationary
    </td>
    <td> 
    Reject < alpha or Accept > alpha
    </td>
</tr>
<tr>
    <td> 
    Alternative Hypothesis
    </td>
    <td> 
    Stationary
    </td>
    <td> 
    Non Stationary
    </td>
    <td> 
    Reject > alpha or Accept < alpha
    </td>
</tr>
</table>

The residuals have a strong autocorrelation with the ACF jumping outside the confidence intervals for several values of lags. So we would need to take seasonal difference on the residuals. The period of seasonality can be determined based on the fact that the original data is obtained from all quarters of the years and shows seasonality of the quarter. This means that the residuals in quarter one of a year is close in magnitude to the residuals from quarter one of the preceding and succeeding years. This observation makes us take differences over periods of four time units as follows:

In [ ]:
# Let's compute quarterly differecing to remove quaterly seasonality
residuals_qtr_diff = residuals.diff(4)

In [ ]:
# Remove null values
residuals_qtr_diff = residuals_qtr_diff.loc[~pd.isnull(residuals_qtr_diff)]

In [ ]:
# Let's plot the autocorrelation function of the residuals
fig = plt.figure(figsize=(5.5, 5.5))
ax = fig.add_subplot(1,1,1)
autocorrelation_plot(residuals_qtr_diff, ax=ax)
ax.set_title('ACF of Quaterly Differenced Residuals')

In [ ]:
# Perform Ljung-Box test on residuals_qtr_diff to get the p-values
# We will use lags of upto 20

# acceptance of null hypothesis confirms stationarity of the time series.

# p-val > alpha Null hypothesis is not rejected means stationary series
# p-val < alpha Null hypothesis is rejected means non-stationary series
_, _, _, pval_residuals = stattools.acf(
    residuals_qtr_diff, unbiased=True, nlags=1, qstat=True, alpha=0.05
)
print('Null hypothesis is rejected for lags:', np.where(pval_residuals<=0.05))

In [ ]:
# Perfom ADF test and check stationary for residuals_qtr_diff
adf_result = stattools.adfuller(residuals_qtr_diff, autolag='AIC')
# alpha is the probability of rejecting the null hypothesis, if it is true.
# p-val > alpha Null hypothesis is not rejected means non-stationary series
# p-val < alpha Null hypothesis is rejected means stationary series

# The null hypothesis is NOT rejected as the p-value is greater than 0.05.

print('p-val of the residuals_qtr_diff:', adf_result[1])